# ArcGIS Online Data


The arcgis Python API allows you to connect to your ArcGIS Online Organization through Python. Just like accessing ArcGIS Online through a web browser, you need to login to access your data.

Data in ArcGIS Online is stored as feature layers and tables. These have similar properties to geodatabases and feature classes. Featuresets and Feature Collections are similar to the feature layers we have worked with in arcpy. These are used to update the feature services.

Link to arcgis Python API documentation: https://developers.arcgis.com/python/api-reference/

### The GIS Module 


The [GIS](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis) object is the way the Python API for ArcGIS handles your credentials to ArcGIS Online.  [This](https://developers.arcgis.com/python/guide/the-gis-module/) page gives a pretty decent overview of the different useful parts of a GIS object.

![Image of API](https://developers.arcgis.com/assets/img/python-graphics/guide_gis_module_01.png)

In [ ]:
import arcgis

session_gis = arcgis.GIS("pro")

In [ ]:
session_gis.users.me

### Items and Feature Layers

The [Item](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#item) object is the basic unit at which ArcGIS Online (and Portal for ArcGIS) tracks data, maps and apps.  

In [ ]:
item_to_export = arcgis.gis.Item(session_gis, '973066d2de4640dfa2c628e884190a35')
item_to_export

The [Feature Layer](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featurelayer) object is the primary way you can interact with data on ArcGIS Online.  Because ArcGIS Online is a REST-based system, the primary way you can pull data from it is with the [query](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.query) method.

In [ ]:
print(item_to_export.layers)

lyr_to_export = item_to_export.layers[0]

In [ ]:
# grab the total amount of features
feature_count = lyr_to_export.query(return_count_only=True)

# check out the fields
fields = [f['name'] for f in lyr_to_export.properties.fields]

print("Total Features: {}".format(feature_count))
for field in fields:
    print(field)

In [ ]:

#The arcgis Python API also supports a map widget, which you can use in Jupyter Notebooks to display maps and data
amap = session_gis.map(location='San Francisco, Ca')
amap

In [ ]:
amap.add_layer(lyr_to_export)

## Exporting ArcGIS Online Data

In [ ]:
df_to_export = lyr_to_export.query(as_df=True)

df_to_export.head()

In [ ]:
df_to_export.drop(columns=['SHAPE']).to_excel('./export.xlsx', index=False)

We can also export a dataframe to an arcpy feature class or shapefile if it has geometry.

### Problem 1 Solution

In [ ]:
import arcpy
import os

gdb = arcpy.management.CreateFileGDB('.', 'AGOL_Export')[0]
df_to_export.spatial.to_featureclass(os.path.join(gdb, 'states'))

## Editing Data in ArcGIS Online


Editing data in ArcGIS Online is a very in-demand process as more workflows are moving to the cloud. While ArcGIS Online makes it much easier for multiple users to edit the same dataset by hand, it presents some problems for automation. With desktop GIS and local datasets, you have pretty easy access to the table data via databases or shapefiles. With ArcGIS Online, you only have access to the server. This means that rather than directly edit the data, you have to interact with the server through the following steps:

- query the target dataset creating a copy
- modify your copy of the dataset
- communicate your changes back to the server

Let's suppose (for an example) that we have a project where we're collecting field data. In this case, we'll use some dummy vegetation data. Our field users want a useful way to identify their points and communicate to each other once they've collected their data.

In desktop programs and databases, ArcGIS provides a special kind of field called an "Auto-Incrementing ID" field. This helps end users communicate which feature they're talking about in a very human-friendly way. A lot of data analysts really like this idea, but ArcGIS Online doesn't do that for us. If we want it, we'll have to do it ourselves.

In [ ]:
item_toedit = arcgis.gis.Item(session_gis, '973066d2de4640dfa2c628e884190a35')
item_toedit

In [ ]:
lyr_toedit = item_toedit.layers[0]

In [ ]:
fset_all = lyr_toedit.query(return_geometry=False)
print(type(fset_all))

df_all = lyr_toedit.query(as_df=True)
print(type(df_all))

In [ ]:
df_all.head()

In [ ]:

where_im_from = 'New York'
fset_homestate = lyr_toedit.query(where="STATE_NAME = '{}'".format(where_im_from))

for feature in fset_homestate.features:
    old_value = feature.get_value('LIVED_IN')
    new_value = old_value + 1
    feature.set_value('LIVED_IN', new_value)

lyr_toedit.edit_features(updates=fset_homestate.features)